**Table of contents**<a id='toc0_'></a>    
1. [Estudio sobre datos de **AirBnB** de la ciudad de Porto y alrededores.](#toc1_)    
1.1. [Importación de librerías, visualización y tratamientos de los dataset](#toc1_1_)    
1.2. [Tratamiento de datos con KNN](#toc1_2_)    
1.2.1. [Tratamiento de outliers](#toc1_2_1_)    
1.3. [Análisis exploratorio](#toc1_3_)    
1.4. [Filtro de distancias](#toc1_4_)    
1.4.1. [Vecindario (aka Freguesia)](#toc1_4_1_)    
1.4.2. [Mapa de la localización de los alojamientos](#toc1_4_2_)    
1.5. [Tipos de propiedades y habitaciones](#toc1_5_)    
1.5.1. [Tipos de habitaciones](#toc1_5_1_)    
1.5.2. [Tipos de propiedades](#toc1_5_2_)    
1.6. [Número de alojados](#toc1_6_)    
1.7. [Analítica al servicio del gobierno.](#toc1_7_)    
1.8. [Consejos al turismo](#toc1_8_)    
1.8.1. [Precio medio por vecindario](#toc1_8_1_)    
1.8.1.1. [Mapa del precio medio por localizaciones](#toc1_8_1_1_)    
1.8.1.2. [Qué áreas de Oporto serán las más rentables para alquilar](#toc1_8_1_2_)    
1.8.1.3. [Mapa de calor con los precios más altos](#toc1_8_1_3_)    
1.8.2. [Seguridad del vecindario](#toc1_8_2_)    
1.8.3. [Review scores location, and location scores versus price](#toc1_8_3_)    
1.9. [Cómo usar las puntuaciones de las opiniones](#toc1_9_)    
1.9.1. [Encontrando un buen hospedador](#toc1_9_1_)    
1.10. [Disponibilidad en el tiempo](#toc1_10_)    
1.10.1. [Precio medio por día](#toc1_10_1_)    
1.11. [Minería de texto con las reviews](#toc1_11_)    
1.12. [Mapa de Porto con AOI](#toc1_12_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Estudio sobre datos de **AirBnB** de la ciudad de Porto y alrededores.](#toc0_)

## 1.1. <a id='toc1_1_'></a>[Importación de librerías, visualización y tratamientos de los dataset](#toc0_)

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from utils.funciones import *

warnings.simplefilter(action="ignore", category=FutureWarning)

# mapas interactivos
import folium
import geopandas as gpd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp

# to make the plotly graphs
import plotly_express as px
from branca.colormap import LinearColormap
from folium.plugins import FastMarkerCluster, HeatMap

# Establecer el tema oscuro como predeterminado
pio.templates.default = "plotly_dark"

from prettymapp.geo import get_aoi
from prettymapp.osm import get_osm_geometries
from prettymapp.plotting import Plot
from prettymapp.settings import STYLES
from scipy import stats


In [ ]:
df_calendar = pd.read_csv('http://data.insideairbnb.com/portugal/norte/porto/2022-12-16/data/calendar.csv.gz', parse_dates=['date'], index_col=['listing_id'])
#df_calendar.to_csv('input/calendar.csv', index=False)

In [ ]:
df_listing_detailed = pd.read_csv('http://data.insideairbnb.com/portugal/norte/porto/2022-12-16/data/listings.csv.gz', index_col= ["id"])
#df_listing_detailed.to_csv('output/listings_detailed.csv', index=False)

In [ ]:
df_reviews = pd.read_csv('http://data.insideairbnb.com/portugal/norte/porto/2022-12-16/data/reviews.csv.gz', parse_dates=['date'])
#df_reviews.to_csv('input/reviews.csv', index=False)

In [ ]:
df_listing = pd.read_csv('http://data.insideairbnb.com/portugal/norte/porto/2022-12-16/visualisations/listings.csv', index_col= ["id"])
#df_listing.to_csv('output/listings.csv', index=False)

In [ ]:
df_listing_detailed['price'] = df_listing_detailed['price'].replace('[$,]', '', regex=True)
df_listing_detailed['price'] = df_listing_detailed['price'].astype(float).round().astype(int)

In [ ]:
df_oporto_review = df_listing # Copio el df aquí para usar las review después en la sección de la Nube de palabras

In [ ]:
df_listing_detailed.head()

In [ ]:
df_listing_detailed.columns

Aquí lo que hago es un left outer join, seguido de un right outer join en el que se excluyan las columnas que ya se han unido en el left join para evitar la duplicación. Con un merge tan solo de pandas no funciona. 

Seleccionamos las columnas con las que vamos a trabajar.

In [ ]:
df_listing.columns


In [ ]:
target_columns = ["accommodates", "host_is_superhost", "host_response_rate", "host_response_time", "listing_url",  "maximum_nights",  "property_type", "review_scores_accuracy", "review_scores_checkin", "review_scores_cleanliness", "review_scores_communication", "review_scores_location", "review_scores_rating", "review_scores_value"]

# "host_about",

In [ ]:
oporto_merge = pd.merge(df_listing, df_listing_detailed[target_columns], on= 'id', how='left')

cols = df_listing.columns

oporto = oporto_merge

#oportopreknn = oporto.to_csv('output/oportopreknn.csv')
# ! Este csv está para ver que una vez se ha hecho el merge ya aparecen los errores en las columnas. A partir de cierto valor parece que una fila se convierte en dos dejando las columnas del final de dicha línea sin rellenar y empezando a rellenar por las columnas del principio de la fila siguiente con las columnas del final de la línea que ha cortado.
oporto.info()

In [ ]:
oporto.isnull().sum()

In [ ]:
plt.figure(figsize=(15, 5))
sns.heatmap(oporto.isnull(), yticklabels=False, cbar=True, cmap="viridis");

In [ ]:
oporto[['host_id', 'price', 'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count',    'availability_365', 'number_of_reviews_ltm', 'accommodates', 'maximum_nights']] = oporto[['host_id', 'price', 'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count',                                                                                                 'availability_365', 'number_of_reviews_ltm', 'accommodates', 'maximum_nights']].astype(int)
oporto['host_response_rate'] = oporto['host_response_rate'].str.replace('%', '').fillna(0) #cambiamos los NaN por 0
oporto['host_response_rate'] = oporto['host_response_rate'].astype(int)
# oporto['last_review'] = pd.to_datetime(oporto['last_review'], format='%Y-%m-%d')
# oporto['first_review'] = pd.to_datetime(oporto['first_review'], format='%Y-%m-%d') 
#! convertir aquí las columnas temporales da error en el KNN hacer después

In [ ]:
oporto.head()

## 1.2. <a id='toc1_2_'></a>[Tratamiento de datos con KNN](#toc0_)

In [ ]:
oporto.duplicated().sum()

In [ ]:
oporto.columns

In [ ]:
df_encoded, encoded_info = fritas(oporto)

In [ ]:
target_column = "price"
bestkar = bravas(df_encoded, target_column)

In [ ]:
bestkar

In [ ]:
df_imputed = impute_missing_values_with_knn(df_encoded, bestkar)

In [ ]:
df_imputed.isnull().sum()

In [ ]:
df_imputed.head()

In [ ]:
oporto_imputed_decode = desfritas(df_imputed, encoded_info)

In [ ]:
oporto_imputed_decode

In [ ]:
plt.figure(figsize=(15, 5))
sns.heatmap(oporto_imputed_decode.isnull(), yticklabels=False, cbar=True, cmap="viridis");

<font color='red'>OJO: muchos de estos valores de la columas de scores están imputados por -1 porque no ha habido forma de que el KNN los lograra rellenar bien o por NaN o NULL. El encoder trata los elementos vacíos y los encodea y luego a la vuelta del decoder esos valores el KNN no los ha tratado. Hay que pensar cómo mejorar la función __fritas__ para que no encodee los valores nulos y así el KNN pueda arreglarlos. <font>

In [ ]:
oporto_imputed_decode.dtypes

In [ ]:
oporto_imputed_decode = pd.DataFrame(oporto_imputed_decode).reset_index()

In [ ]:
#oporto_imputed_decode.to_csv("output/oporto_test_trimmed.csv", index=True, compression='infer', header=True, chunksize=10000, encoding='UTF-16', date_format=None)
# ? No sé ni qué hace esto, Demetrio...

In [ ]:
oporto_imputed_decode

In [ ]:
oporto_imputed_decode['latitude'].sort_values(ascending=False)

### 1.2.1. <a id='toc1_2_1_'></a>[Tratamiento de outliers](#toc0_)

In [ ]:
oporto_imputed_decode=oporto_imputed_decode[(np.abs(stats.zscore(oporto_imputed_decode['price'])) < 3)]

## 1.3. <a id='toc1_3_'></a>[Análisis exploratorio](#toc0_)

## 1.4. <a id='toc1_4_'></a>[Filtro de distancias](#toc0_)

La fórmula de Haversine es una fórmula matemática que se utiliza para calcular la distancia entre dos puntos en la superficie de una esfera, como la Tierra. La fórmula se basa en la longitud y la latitud de los dos puntos, y utiliza la ley de los cosenos para calcular la distancia entre ellos.

La fórmula de Haversine se puede expresar de la siguiente manera:

$$d = 2 R * \arcsin\left(\sqrt{\sin^2\left(\frac{lat_2-lat_1}{2}\right) + \cos(lat_1)\cos(lat_2)\sin^2\left(\frac{lon_2-lon_1}{2}\right)}\right)$$

donde:

- d es la distancia entre los dos puntos en la superficie de la esfera
- R es el radio de la esfera (en metros, kilómetros, millas, etc.)
- $lat_1$ y $lat_2$ son las latitudes de los dos puntos (en radianes o grados)
- $lon_1$ y $lon_2$ son las longitudes de los dos puntos (en radianes o grados)

In [ ]:
df_distancia = filtrar_por_distancia(oporto_imputed_decode, 20)

In [ ]:
df_55 = filtrar_por_distancia(oporto_imputed_decode,55)
df_55.to_csv('output/df_55.csv', index= True)

#* Este df va a servir para pasarlo a streamlit y en vez de pasar el oporto y la función de filtro por distancias directamente con el slider se seleccionarán los elementos del dataset con la distancia < a la fijada en el slider.

In [ ]:
df_distancia

In [ ]:
df_distancia['distancia'].describe().T

In [ ]:
df_distancia.head()

In [ ]:
df_distancia.dtypes

In [ ]:
df_distancia.count()

In [ ]:
correlation_papa(df_distancia, annot=False)

### 1.4.1. <a id='toc1_4_1_'></a>[Vecindario (aka Freguesia)](#toc0_)

In [ ]:
feq = oporto['neighbourhood'].value_counts().sort_values(ascending=True)
feq = feq[feq>500]

fig = px.bar(feq, x=feq.values, y=feq.index, orientation='h')
fig.update_layout(
    title="Number of listings by neighbourhood",
    xaxis_title="Number of listings",
    yaxis_title="Neighbourhood",
    font=dict(size=12)
)
fig.show()

La Unión de las Parroquias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória, también conocida como Unión de Parroquias del Centro Histórico de Oporto, es una parroquia portuguesa del municipio de Oporto, creada por la Ley n.º 11-A/2013 de 28 de enero, que une las antiguas parroquias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau y Vitória.

Tiene una superficie total de 5,43 km², una población de 37.436 habitantes (2021) y una densidad de población de 7.447,5 habitantes por km².

En resumen, la Unión de Parroquias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória es una entidad administrativa portuguesa que se creó en 2013, que unió seis antiguas parroquias y ahora se conoce como la Unión de Parroquias del Centro Histórico de Oporto. La parroquia tiene una población de alrededor de 37.436 personas y una densidad de población bastante alta de 7.447,5 habitantes por km².

La palabra "freguesia" proviene del latín "filium ecclesiae", que significa "hijo de la iglesia". En Portugal, el antiguo Imperio Portugués y el Imperio de Brasil, una freguesia es la unidad administrativa más pequeña y obligatoria de los municipios, similar a una parroquia civil en otros países. Cada municipio tiene al menos una freguesia, y en algunos casos el territorio de una freguesia coincide con el del municipio.

### 1.4.2. <a id='toc1_4_2_'></a>[Mapa de la localización de los alojamientos](#toc0_)

In [ ]:
lats = df_distancia['latitude'].tolist()
lons = df_distancia['longitude'].tolist()
locations = list(zip(lats, lons)) #Guardamos latitudes y longitudes, hacemos una tupla y las cambiamos a una lista.

map1 = folium.Map(location=[41.1496, -8.6109], zoom_start=12) # Le das una lat y lon inicial y un zoom inicial para representar el mapa
FastMarkerCluster(data=locations).add_to(map1) # Te añade las localizaciones al mapa generado anteriormente
folium.Marker(location=[41.1496, -8.6109]).add_to(map1)
map1

## 1.5. <a id='toc1_5_'></a>[Tipos de propiedades y habitaciones](#toc0_)

### 1.5.1. <a id='toc1_5_1_'></a>[Tipos de habitaciones](#toc0_)

Buscar info sobre ley en oporto 

_El tipo de habitación es muy importante en Ámsterdam, porque Ámsterdam tiene la regla de que las casas/apartamentos completos solo se pueden alquilar a través de Airbnb por un máximo de 60 días al año. A continuación, podemos ver que esta restricción se aplica a la mayoría de los listados._

In [ ]:
freq = df_distancia['room_type'].value_counts().sort_values(ascending=True)

fig = px.bar(freq, orientation='h', color=freq.index,
             labels={'y': 'Room Type', 'x': 'Number of Listings'})
fig.update_layout(title="Number of Listings by Room Type",
                  xaxis_title="Number of Listings",
                  yaxis_title="Room Type",
                  height=400, width=800)
fig.show()

### 1.5.2. <a id='toc1_5_2_'></a>[Tipos de propiedades](#toc0_)

En el conjunto de datos, encontramos muchos tipos de propiedades diferentes.

In [ ]:
df_distancia['property_type'].unique()

_Sin embargo, muchos de esos tipos de propiedades tienen muy pocos listados en Ámsterdam. En la figura a continuación, solo mostramos tipos de propiedades con al menos 100 listados. Como podemos ver, la gran mayoría de las propiedades en Ámsterdam son apartamentos._

In [ ]:
prop = df_distancia.groupby(['property_type','room_type']).room_type.count()
prop = prop.unstack()
prop['total'] = prop.iloc[:,0:3].sum(axis = 1)
prop = prop.sort_values(by=['total'])
prop = prop[prop['total']>=100]
prop = prop.drop(columns=['total'])

fig = px.bar(prop, barmode='stack', orientation='h',
             color_discrete_sequence=["rgb(255, 102, 102)", "rgb(102, 178, 255)", "rgb(102, 255, 178)"],
             width=1000, height=600)
fig.update_layout(title='Property types in Oporto', xaxis_title='Number of listings', yaxis_title='',
                  legend_title='', font=dict(size=14))
fig.show()

## 1.6. <a id='toc1_6_'></a>[Número de alojados](#toc0_)
_
Como era de esperar, la mayoría de los listados son para 2 personas. Además, Airbnb utiliza un máximo de 16 huéspedes por anuncio._

In [ ]:
feq = df_distancia['accommodates'].value_counts().sort_index().reset_index()
feq.columns = ['Accommodates', 'Number of listings']
fig = px.bar(feq, x='Accommodates', y='Number of listings', 
             color='Accommodates',
             width=700, height=500)
fig.update_layout(title={'text':"Accommodates (number of people)", 'x':0.5},
                  xaxis_title='Accommodates', yaxis_title='Number of listings',
                  font=dict(size=14))
fig.show()

_Sin embargo, Ámsterdam tiene una restricción adicional. Debido a las consideraciones de riesgo de incendio y también teniendo en cuenta un posible grupo ruidoso, los propietarios solo pueden alquilar su propiedad a grupos con un máximo de 4 personas. ¡Esto realmente significa que las listas que indican que el número máximo de personas es superior a 4 están infringiendo esta regla!_

In [ ]:
df_distancia[df_distancia['accommodates']>15].head()

## 1.7. <a id='toc1_7_'></a>[Analítica al servicio del gobierno.](#toc0_)

## 1.8. <a id='toc1_8_'></a>[Consejos al turismo](#toc0_)

### 1.8.1. <a id='toc1_8_1_'></a>[Precio medio por vecindario](#toc0_)

_Para comparar "manzanas con manzanas" solo seleccionaremos el tipo de alojamiento más común, que es el alojamiento para 2 personas. Como era de esperar, el alojamiento en el centro de la ciudad es el más caro._

In [ ]:
feq = df_distancia[df_distancia['accommodates']==2]
feq = feq.groupby('neighbourhood')['price'].mean().sort_values(ascending=True)

fig = px.bar(feq, orientation='h', width=800, height=1200, color=feq.values,
             color_continuous_scale='RdYlGn_r', labels={'y':'Neighbourhood', 'x':'Average daily price (Euro)'},
             title='Average daily price for a 2-persons accommodation', template='plotly_dark')

fig.update_layout(template='plotly_dark')
fig.show()

#### 1.8.1.1. <a id='toc1_8_1_1_'></a>[Mapa del precio medio por localizaciones](#toc0_)

In [ ]:
# Read in data
porto_geojson = "http://data.insideairbnb.com/portugal/norte/porto/2022-12-16/visualisations/neighbourhoods.geojson"
porto_gdf = gpd.read_file(porto_geojson)


In [ ]:

# Calculate mean price by neighborhood for listings that accommodate 2 people
mean_prices = df_distancia.loc[df_distancia['accommodates'] == 2].groupby('neighbourhood')['price'].mean()

# Join the mean prices to the geojson
porto_gdf = porto_gdf.join(mean_prices, on='neighbourhood')

# Drop neighborhoods without mean prices
porto_gdf.dropna(subset=['price'], inplace=True)

# Round the mean prices and create a dictionary for the color map
price_dict = porto_gdf.set_index('neighbourhood')['price'].round().to_dict()

# Define color map
color_scale = LinearColormap(['green', 'yellow', 'red'], vmin=min(price_dict.values()), vmax=max(price_dict.values()), caption='Average price')

# Define style and highlight functions
def style_function(feature):
    return {
        'fillColor': color_scale(price_dict.get(feature['properties']['neighbourhood'], 0)),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.5
    }

def highlight_function(feature):
    return {
        'weight': 3,
        'fillColor': color_scale(price_dict.get(feature['properties']['neighbourhood'], 0)),
        'fillOpacity': 0.8
    }

# Create map
map3 = folium.Map(location=[41.1496, -8.6109], zoom_start=11)

# Add geojson layer to map with tooltip and style and highlight functions
folium.GeoJson(
    data=porto_gdf,
    name='Oporto',
    tooltip=folium.features.GeoJsonTooltip(fields=['neighbourhood', 'price'], labels=True, sticky=False),
    style_function=style_function,
    highlight_function=highlight_function
).add_to(map3)

# Add marker to map
folium.Marker(location=[41.1496, -8.6109]).add_to(map3)

# Add color scale to map
map3.add_child(color_scale)





# contador = 0 
# "tu bucle for"
# contador += 1
# if contador > 99 #pordeciralgo:
#      print(f"límite alcanzado")
#      break 

#### 1.8.1.2. <a id='toc1_8_1_2_'></a>[Qué áreas de Oporto serán las más rentables para alquilar](#toc0_)

Basándonos en las localizaciones más turísticas de la ciudad de Oporto haremos una correlación con el precio para poder determinar cuáles son las zonas más rentables para tener un Airbnb.

#### 1.8.1.3. <a id='toc1_8_1_3_'></a>[Mapa de calor con los precios más altos](#toc0_)

In [ ]:
# Mapa de calor basándome en uno de Demetrio

# Get the minimum and maximum price values
min_price = df_distancia['price'].min()
max_price = df_distancia['price'].max()
# Define the color scale for the legend
color_scale = LinearColormap(['green', 'yellow', 'red'], vmin=min_price, vmax=max_price, caption='Precio')




# Create the map
calorsita = folium.Map(location=[41.1496, -8.6109], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=df_distancia[['latitude', 'longitude', 'price']],
        radius=20,
        gradient={0.2: 'green', 0.5: 'yellow', 1: 'red'},
        min_opacity=0.2).add_to(calorsita)

# Add the color scale legend
calorsita.add_child(color_scale)


# Display the map
calorsita


### 1.8.2. <a id='toc1_8_2_'></a>[Seguridad del vecindario](#toc0_)

### 1.8.3. <a id='toc1_8_3_'></a>[Review scores location, and location scores versus price](#toc0_)

En esta sección, agrupamos los puntajes de revisión de la ubicación por vecindario (solo listados con al menos 10 revisiones). Aunque esperamos que la distancia al centro de la ciudad sea un factor importante, esta puntuación también debería tener en cuenta otras cosas. Otros factores pueden incluir:

* La seguridad de una ubicación (como se muestra en la sección anterior)
* Ruido. Si una lista tiene una ubicación central, pero está rodeada de bares ruidosos, eso debería costar puntos en el puntaje de revisión de la ubicación.
* Si un listado está ubicado fuera del centro de la ciudad pero bien conectado por transporte público, debería obtener puntos de bonificación por eso.
* Instalaciones cercanas al listado. ¿Hay supermercados, bares y restaurantes cerca?
* Puede que algunas personas busquen aparcamiento gratuito si vienen en coche (el aparcamiento es muy caro en Ámsterdam en general).

A continuación, vemos que los vecindarios centrales, que generalmente también fueron los más caros, generalmente también obtienen una puntuación más alta en la puntuación de revisión de ubicación. Si calculara la distancia al centro de la ciudad para cada listado, espero ver correlaciones bastante fuertes entre esta distancia con el puntaje de revisión del precio y la ubicación.

Al mirar el puntaje promedio de revisión, me sorprende ver que el promedio está por encima de 8/10 para todos los vecindarios. Ámsterdam es una ciudad pequeña (¡mucho más pequeña de lo que mucha gente piensa!). Por lo tanto, no lleva mucho tiempo llegar al centro de la ciudad desde cualquier lugar, lo que podría explicar hasta cierto punto los altos promedios. Mi consejo personal para los turistas sería considerar un alojamiento más asequible fuera del centro de la ciudad, en un vecindario seguro y con buenas conexiones de transporte público al centro de la ciudad de todos modos. Sin embargo, ¿las diferencias entre las mejores ubicaciones y los vecindarios exteriores son realmente tan pequeñas? ¡Vamos a averiguarlo en la siguiente sección!


In [ ]:
# Group by neighbourhood and calculate the mean review score location for listings with at least 10 reviews
feq1 = df_distancia[df_distancia['number_of_reviews'] >= 10].groupby('neighbourhood')['review_scores_location'].mean().sort_values(ascending=True)

# Create bar chart using Plotly Express
fig1 = px.bar(feq1, x='review_scores_location', y=feq1.index, orientation='h', color='review_scores_location', color_continuous_scale='RdYlGn')
fig1.update_layout(xaxis_title="Score (scale 1-10)", yaxis_title="") 

# Group by neighbourhood and calculate the mean daily price for 2-person accommodations
feq2 = df_distancia[df_distancia['accommodates'] == 2].groupby('neighbourhood')['price'].mean().sort_values(ascending=True)

# Create bar chart using Plotly Express
fig2 = px.bar(feq2, x='price', y=feq2.index, orientation='h', color='price', color_continuous_scale='viridis')
fig2.update_layout(xaxis_title="Average daily price (Euro)", yaxis_title="")

# Combine the two charts into a single subplot
figures = sp.make_subplots(rows=2, cols=1, subplot_titles=("Average review score location (at least 10 reviews)", "Average daily price for a 2-persons accommodation"))

# Add each bar chart to the subplot
figures.add_trace(fig1['data'][0], row=1, col=1)
figures.add_trace(fig2['data'][0], row=2, col=1)

# Update the layout of the subplot
figures.update_layout(
    title_text="Locations",
    height=800,
    width=1000,
    font_size=12,
    showlegend=False
)
# TODO CUIDADO CON LA BARRA DE COLORES

## 1.9. <a id='toc1_9_'></a>[Cómo usar las puntuaciones de las opiniones](#toc0_)

Además de las reseñas escritas, los invitados pueden enviar una calificación de estrellas general y un conjunto de calificaciones de estrellas de categoría. Los huéspedes pueden dar calificaciones sobre:

* Experiencia general. ¿Cuál fue su experiencia en general?
* Limpieza. ¿Sentiste que tu espacio estaba limpio y ordenado?
* Precisión. ¿Con qué precisión su página de listado representó su espacio?
* Valor. ¿Sintió que su listado proporcionó un buen valor por el precio?
* Comunicación. ¿Qué tan bien se comunicó con su anfitrión antes y durante su estadía?
* Llegada. ¿Qué tan bien fue su registro?
* Ubicación. ¿Cómo te sentiste en el barrio?

A continuación puede ver la distribución de puntajes de todas esas categorías. ¡Lo que me llamó la atención de inmediato es que las puntuaciones parecen realmente altas en todos los ámbitos!. Está bien explicado en este artículo: [¿Más alta que la calificación promedio? El 95 % de los listados de Airbnb calificaron de 4,5 a 5 estrellas](https://mashable.com/2015/02/25/airbnb-reviews-above-average/?europe=true#1YLfzOC34sqd).

Después de haber visto las distribuciones de puntajes, personalmente consideraría que cualquier puntaje de 8 o inferior no es un buen puntaje.

In [ ]:
df_distancia.shape

In [ ]:
# Select listings with at least 10 reviews
listings10 = df_distancia[df_distancia['number_of_reviews']>=10]

# Create histogram figures for each review category
fig1 = px.histogram(listings10, x='review_scores_location',
             barmode='group', category_orders={'review_scores_location': sorted(listings10['review_scores_location'].unique())})
fig1.update_layout(title="Location", xaxis_title="Average review score", yaxis_title="Number of listings", font_size=14)

fig2 = px.histogram(listings10, x='review_scores_cleanliness',
              barmode='group', category_orders={'review_scores_cleanliness': sorted(listings10['review_scores_cleanliness'].unique())})
fig2.update_layout(title="Cleanliness", xaxis_title="Average review score", yaxis_title="Number of listings", font_size=14)

fig3 = px.histogram(listings10, x='review_scores_value',
              barmode='group', category_orders={'review_scores_value': sorted(listings10['review_scores_value'].unique())})
fig3.update_layout(title="Value", xaxis_title="Average review score", yaxis_title="Number of listings", font_size=14)

fig4 = px.histogram(listings10, x='review_scores_communication',
              barmode='group', category_orders={'review_scores_communication': sorted(listings10['review_scores_communication'].unique())})
fig4.update_layout(title="Communication", xaxis_title="Average review score", yaxis_title="Number of listings", font_size=14)

fig5 = px.histogram(listings10, x='review_scores_checkin',
              barmode='group', category_orders={'review_scores_checkin': sorted(listings10['review_scores_checkin'].unique())})
fig5.update_layout(title="Arrival", xaxis_title="Average review score", yaxis_title="Number of listings", font_size=14)

fig6 = px.histogram(listings10, x='review_scores_accuracy',
              barmode='group', category_orders={'review_scores_accuracy': sorted(listings10['review_scores_accuracy'].unique())})
fig6.update_layout(title="Accuracy", xaxis_title="Average review score", yaxis_title="Number of listings", font_size=14)

# Create subplot with 2 rows and 3 columns, with titles for each subplot
figs = sp.make_subplots(rows=2, cols=3, subplot_titles=("Location", "Cleanliness", "Value", "Communication", "Arrival", "Accuracy"))

# Add each bar chart to the subplot
figs.add_trace(fig1['data'][0], row=1, col=1)
figs.add_trace(fig2['data'][0], row=1, col=2)
figs.add_trace(fig3['data'][0], row=1, col=3)
figs.add_trace(fig4['data'][0], row=2, col=1)
figs.add_trace(fig5['data'][0], row=2, col=2)
figs.add_trace(fig6['data'][0], row=2, col=3)

# Update layout for the subplot
figs.update_layout(
    title_text="Review Scores",
    height=800,
    width=1000,
    font_size=12,
    showlegend=False
)
# TODO INTENTAR PONER EL COLOR DE LAS REVIEWS
# Show

### 1.9.1. <a id='toc1_9_1_'></a>[Encontrando un buen hospedador](#toc0_)



En Airbnb puedes obtener el estatus de "Superhost". De Airbnb:
* Como SuperAnfitrión, tendrá más visibilidad, potencial de ingresos y recompensas exclusivas. Es nuestra manera de decir gracias por su hospitalidad excepcional.
* Cómo convertirse en Superhost: cada 3 meses, verificamos si cumple con los siguientes criterios. Si lo haces, ganarás o mantendrás tu estatus de SuperAnfitrión.
    * Los Superanfitriones tienen una calificación general promedio de 4.8 o superior según las reseñas de al menos el 50 % de sus huéspedes de Airbnb durante el último año.
    * Los Superhosts han alojado al menos 10 estadías en el último año o, si realizan reservas a más largo plazo, 100 noches en al menos 3 estadías.
    * Los Superhosts no tienen cancelaciones en el último año, a menos que haya circunstancias atenuantes.
    * Los Superhosts responden al 90 % de los mensajes nuevos en 24 horas.

A continuación, podemos ver que solo una pequeña parte de los listados en Ámsterdam tienen un anfitrión que es Superanfitrión.

In [ ]:
df_frequencies = df_distancia['host_is_superhost'].value_counts(normalize=True).reset_index()
df_frequencies.columns = ['Superhost', 'Percentage']
df_frequencies['Percentage'] = df_frequencies['Percentage'] * 100

fig = px.bar(df_frequencies, x='Superhost', y='Percentage',
             labels={'Superhost': 'Superhost', 'Percentage': 'Percentage (%)'},
             color='Superhost',
             color_discrete_map={'f': 'rgb(255, 0, 0)', 't': 'rgb(0, 128, 0)'})

fig.update_traces(texttemplate='%{y:.2f}%', textposition='inside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(legend_title='Superhost', legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig.update_layout(
    title_text="Percentage of Superhost",
    height=400,
    width=1000,
    font_size=12,
    showlegend=False
)

Si tuviéramos que reservar alojamiento, no buscaríamos necesariamente un superhost. En realidad, me temo que pagaría demasiado, ya que el superhost probablemente aumentará sus precios. Sin embargo, tampoco me gustaría un host que responde mal o cancela mucho.

Como podemos ver, más de 5.000 de los 20.000 listados tienen al menos 10 reseñas y responden al menos al 90% de los mensajes nuevos. Consideraría que esos anfitriones son buenos respondedores "probados" (lo que no significa que una lista con menos de 10 reseñas no pueda tener buenos anfitriones que respondan; simplemente no está probado todavía). Además, hay muy pocos listados con anfitriones que no respondan a los mensajes nuevos dentro de las 24 horas.

In [ ]:
listings10 = df_distancia[df_distancia['number_of_reviews'] >= 10]
feq1 = listings10['host_response_rate'].replace(0, np.nan).dropna().sort_values(ascending=True) #Cambiamos los valores 0 con los que rellenamos al principio por nan para poder droppearlos

fig1 = px.histogram(feq1, nbins=35)
fig1.update_layout(title='Response rate (at least 10 reviews)', xaxis_title='Percent', yaxis_title='Number of Listings', font=dict(size=20))

#TODO Arreglar el gráfico 1 para que los límites de los ejes tengan sentido.

response_time_counts = listings10.dropna(subset=['host_response_time'])['host_response_time'].value_counts().reset_index()
response_time_counts.columns = ['response_time', 'count']

fig2 = px.bar(response_time_counts, x='response_time', y='count', labels={'response_time': 'Response time', 'count': 'Number'})
fig2.update_layout(title='Response time (at least 10 reviews)', xaxis_title_font_size=20, 
                   yaxis_title_font_size=20, font=dict(size=16))


# Create subplot with 2 rows and 3 columns, with titles for each subplot
figs = sp.make_subplots(rows=1, cols=2, subplot_titles=("Response rate", "Response time"))

# Add each bar chart to the subplot
figs.add_trace(fig1['data'][0], row=1, col=1)
figs.add_trace(fig2['data'][0], row=1, col=2)

# Update layout for the subplot
figs.update_layout(
    title_text="Responses",
    height=400,
    width=1000,
    font_size=12,
    showlegend=False, 
    template = 'plotly_dark'
)


## 1.10. <a id='toc1_10_'></a>[Disponibilidad en el tiempo](#toc0_)

El archivo de calendario contiene 365 registros para cada listado, lo que significa que para cada listado, el precio y la disponibilidad por fecha se especifican con 365 días de anticipación.

In [ ]:
df_calendar.price = df_calendar.price.str.replace(",","")
df_calendar['price'] = pd.to_numeric(df_calendar['price'].str.strip('$'))
#df_calendar = df_calendar[df_calendar.date < '2023-1-1']

In [ ]:
print(df_calendar.shape)

In [ ]:
df_calendar.head()

In [ ]:
df_distancia.head()

A continuación se muestra un ejemplo de los datos del calendario. Importante tener en cuenta: la disponibilidad es FALSE significa que el propietario no quiere alquilar su propiedad en la fecha específica o que la lista ya se ha reservado para esa fecha. Como queremos comparar manzanas con manzanas nuevamente con respecto a los precios en la siguiente sección, estamos fusionando la variable 'accomodate' con el calendario.

In [ ]:
df_calendar = pd.merge(df_distancia, df_calendar, left_on='id', right_index=True)

In [ ]:
df_calendar.head()

In [ ]:
df_calendar.columns

In [ ]:
df_cal = df_calendar[['date', 'price_x', 'available', 'accommodates']]

In [ ]:
df_cal.to_csv('output/df_cal.csv.gz', compression='gzip')

A continuación, vemos que hasta tres meses por delante, generalmente hay más alojamientos disponibles que en el futuro. Las razones de esto pueden ser que los anfitriones están actualizando más activamente sus calendarios en este período de tiempo. Este gráfico es **interactivo** y, al pasar el cursor sobre los puntos, se mostrará una información sobre herramientas con el "número de listados disponibles" y el "día de la semana" por fecha.

In [ ]:

# Leer los datos y convertir la columna 'date' en tipo datetime
df_calendar['date'] = pd.to_datetime(df_calendar['date'])
# Filtrar los datos para tener sólo los disponibles
sum_available = df_calendar[df_calendar.available == "t"].groupby(['date']).size().to_frame(name= 'available').reset_index()

# Agregar la columna de día de la semana
sum_available['weekday'] = sum_available['date'].dt.day_name()

# Establecer 'date' como el índice del DataFrame
sum_available = sum_available.set_index('date')

# Crear la figura de Plotly Express
fig = px.line(sum_available, y='available', title='Number of listings available by date', template = 'plotly_dark')

# Mostrar la figura
fig.show()

### 1.10.1. <a id='toc1_10_1_'></a>[Precio medio por día](#toc0_)



A continuación, verá el precio promedio de todos los alojamientos para 2 personas marcados como disponibles por fecha. El pico del precio promedio de 240 euros es el 31 de diciembre y el patrón cíclico se debe a precios más altos en los fines de semana. Sin embargo, sospecho que los precios para fechas más lejanas en el tiempo aún no están actualizados y probablemente sean precios predeterminados. Esto podría dar lugar a que el anfitrión no acepte una reserva si se da cuenta de que alguien está intentando reservar algo en una fecha que debería haber sido más cara de lo habitual. Este gráfico es **interactivo** y, al pasar el cursor sobre los puntos, se mostrará una información sobre herramientas con el precio promedio y el día de la semana por fecha.

In [ ]:
df_calendar

In [ ]:
numeric_columns = df_calendar.select_dtypes(include=[np.number]).columns
average_price = df_calendar[(df_calendar.available == "t") & (df_calendar.accommodates == 2)].groupby(['date'])[numeric_columns].mean().astype(np.int64).reset_index()
average_price['weekday'] = average_price['date'].dt.day_name()
average_price = average_price.set_index('date')

In [ ]:
fig = px.line(average_price, x=average_price.index, y='price_x', title='Average price of available 2 persons accommodation by date')
fig.update_traces(text=average_price['weekday'])
fig.update_layout(xaxis_title='Date', yaxis_title='Price', template = 'plotly_dark')
fig.show()

In [ ]:
df_distancia.to_csv('output/df_distancia.csv', index= True)

## 1.11. <a id='toc1_11_'></a>[Minería de texto con las reviews](#toc0_)

Veamos ahora cómo podríamos obtener algo de información extra de las opiniones de los usuarios, en términos muy elementales. Esta sección es un preliminar muy introductorio y básico de un amplio área de estudio conocida como *topic modelling*.

El archivo de "reviews" resultó no ser muy interesante, ya que solo contiene fechas de revisión para cada listado, lo que significa que solo es bueno para contar el número de revisiones. El archivo "reviews_details" contiene la misma información (y la misma cantidad de registros), con 4 columnas adicionales. Además, fusionamos host_id y host_names de la lista con el archivo reviews_details.

In [ ]:
df_reviews = pd.merge(df_reviews, df_oporto_review[['host_id', 'host_name', 'name']], left_on = "listing_id", right_index=True, how = "left")
df_reviews = df_reviews.set_index('id')
df_reviews = df_reviews[['listing_id', 'name', 'host_id', 'host_name', 'date', 'reviewer_id', 'reviewer_name', 'comments']]
df_reviews

In [ ]:
host_reviews = df_reviews.groupby(['host_id', 'host_name']).size().sort_values(ascending=False).to_frame(name = "number_of_reviews")
host_reviews.head()

In [ ]:
df_reviews.comments.head()

## 1.12. <a id='toc1_12_'></a>[Mapa de Porto con AOI](#toc0_)

In [ ]:
aoi = get_aoi(address="Porto, Portugal", radius=1200, rectangular=False)
df = get_osm_geometries(aoi=aoi)

fig = Plot(
    df=df,
    aoi_bounds=aoi.bounds,
    draw_settings=STYLES["Peach"]
).plot_all()

fig.savefig("img/map.jpg")